In [31]:
from langchain.agents import load_tools, AgentType
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
import os
knowledge="简易方程"
os.environ["SERPAPI_API_KEY"] = "PUT YOUR SERPAPI_API_KEY HERE"
os.environ["OPENAI_API_KEY"] = "PUT YOUR OPENAI-API-KEY available to GPT4 HERE"
# llm
openai_api_key = os.environ["OPENAI_API_KEY"]
llm = ChatOpenAI(temperature=0.5, openai_api_key=openai_api_key, model_name="gpt-4-turbo-preview")
# 谷歌搜索 key
serpapi_api_key = os.environ["SERPAPI_API_KEY"]
toolkit = load_tools(["serpapi"], llm=llm, serpapi_api_key=serpapi_api_key)
agent = initialize_agent(
    tools=toolkit,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

template = "{value}，请使用中文回答"
prompt = PromptTemplate(
    input_variables=["value"],
    template=template,
)

questions=f"请帮我找几道中小学{knowledge}的典型应用题，并总结一下常见题型"
print("回答结果: \n")
res = agent.run(prompt.format(value=questions))
print(res)


回答结果: 



> Entering new AgentExecutor chain...
这类问题通常涉及到基础的数学知识，常见的题型包括但不限于：物品的购买问题、速度和时间的问题、年龄问题、工作量问题等。这些问题的解法通常涉及到建立一个或多个方程来解决实际问题。我需要搜索一些具体的例题来更好地回答这个问题。
Action: Search
Action Input: 中小学简易方程应用题典型例题
Observation: ['【解题思路和方法】 简单的题目可以直接套用公式；复杂的题目变通后再用公式。 示例甲乙两班共有学生98人，甲班比乙班多6人，求两班各有多少人？ 解甲 ...', '【解题思路和方法】 简单的题目可以直接套用公式；复杂的题目变通后再用公式。 例1 、甲乙两班共有学生98人，甲班比乙班多6人，求两班各有多少人？', '小学数学中把含有数量关系的实际问题用语言或文字叙述出来，这样所形成的题目叫做应用题。任何一道应用题都由两部分构成。第一部分是已知条件（简称条件） ...', '小学1-6年级奥数题及答案每日一练2022.7.1（每日都做养成好习惯）-283; 小学五年级下册数学暑假天天练习（第一周习题）-441; 小学数学典型应用题（牛吃 ...', '二、以总量为等量关系建立方程例题甲、乙两个粮仓一共有粮6800包，甲是乙的3倍，两仓各有多少包? 解设：乙仓有粮X包，那么甲仓有粮3X包甲粮仓的包数+乙粮仓的包数=总共 ...', '小学数学30种典型应用题讲解 · 1、审题，明白题目是什么意思，根据条件找数量关系 · 2、列式解答，可以用方程，也可以用算术 · 3、最后做答.', '新增加一些代数初步知识，如不等式，用字母表示数，求代数式的值，简易方程，列方程解应用题等。 ... 应用题教学是小学数学教学中的重点和难点，在小学数学中占有十分重要的 ...', 'a 审题理解题意：了解应用题的内容，知道应用题的条件和问题。读题时，不丢字不添字边读边思考，弄明白题中每句话的意思。也可以复述条件和问题，帮助理解 ...', '... 题目中的已知条件，按照等量关系列出方程。?（4）解；求出所列方程的解 ... 小学数学最典型的30道应用题：定义数量关系例题详解 · 期中考试，小学数学 ...', '2016-08-08·小学四年级方程解应用题知识点及

In [32]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0.5,model_name="gpt-4-turbo-preview")


# 定义输出格式为问题列表
class QuestionList(BaseModel):
    question_contents: List[str] = Field(description="list of question contents")

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=QuestionList)

template = """你是一名数学教研专家，需要编写一些适合小学同学练习的数学题,{format_instructions}
             我需要你为我编写 {cnt} 个关于 {knowledge}知识的问题, 
             参考现有的题目{res}，
             难度为 {difficulty}，{otherRestriction}，只需要给出问题即可"""
prompt = PromptTemplate(
    template=template, 
    input_variables=["cnt", "knowledge","difficulty","otherRestriction","res"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

ans=chain.invoke({"cnt":3,
                  "knowledge":knowledge,
                  "difficulty":"简单",
                  "otherRestriction":"你需要为题目编一个完整、有教育意义的故事背景，创设的题目需要以生活中真实发生的问题为背景，使用在生活中会真实出现的数字，问题是要解决生活中实际出现的问题",
                  "res":res})
print(ans)

{'question_contents': ['在一次学校运动会上，男生和女生共参加了100米赛跑项目。如果女生人数是男生的2倍，而且总共有90人参加，那么男生和女生各有多少人参加了100米赛跑？', '小明和小红的储蓄罐里一共有360元。如果小明的钱是小红的3倍，那么小明和小红各有多少钱在他们的储蓄罐里？', '一个家庭在周末计划去超市购物，他们计划购买水果和蔬菜。如果水果的总价是蔬菜的4倍，而且他们一共花了500元，那么水果和蔬菜各花了多少钱？']}


In [34]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain_experimental.tools import PythonREPLTool
# First, let's load the language model we're going to use to control the agent.
llm = ChatOpenAI(temperature=0.5, openai_api_key=openai_api_key, model_name="gpt-4-turbo-preview")
 
# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)
 
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
 

#初始化存储问题和答案的列表
question_with_answer=[]
for question in ans["question_contents"]:
    solution=agent.run(question+"返回解题过程和答案，计算时只能提供单行的python数学表达式，这样子的表达式无法被计算'470 // 60, 470 % 60'，不能出现未定义的符号，如果有多个要计算的表达式请你逐步计算")
    question_with_answer.append({"content":question,"description":solution})
    print(question_with_answer)




> Entering new AgentExecutor chain...
To solve this problem, we can set up the equation based on the given information. Let x be the number of boys. Since the number of girls is twice the number of boys, we can represent the number of girls as 2x. The total number of participants is 90, so the equation to represent this situation is x + 2x = 90. Simplifying this equation will give us the number of boys (x), and then we can easily find the number of girls (2x).

Action: Calculator
Action Input: (90 / 3)
Observation: Answer: 30.0
Thought:I now know the number of boys is 30. To find the number of girls, I can multiply the number of boys by 2.

Action: Calculator
Action Input: 30 * 2
Observation: Answer: 60
Thought:I now know the final answer.
Final Answer: 男生有30人参加了100米赛跑，女生有60人参加了100米赛跑。

> Finished chain.
[{'content': '在一次学校运动会上，男生和女生共参加了100米赛跑项目。如果女生人数是男生的2倍，而且总共有90人参加，那么男生和女生各有多少人参加了100米赛跑？', 'description': '男生有30人参加了100米赛跑，女生有60人参加了100米赛跑。'}]


> Entering new AgentExecutor chain...

In [35]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
model = ChatOpenAI(temperature=0.5,model_name="gpt-4-turbo-preview")

# 定义输出格式为问题列表
class Question(BaseModel):
    content: str = Field(description="问题内容")
    choices: List[str]=Field(description="题目选项数组，一般为3个，选项内容不需要包含ABCD")
    description:str=Field(description="题目解析")
    correct_answer:str=Field(description="正确答案选项")

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Question)

template = """现在为你提供针对小初学生的数学问题和答案，请你把其改造为选择题。问题为：{question},答案为：{answer},
            {format_instructions}
            """
prompt = PromptTemplate(
    template=template, 
    input_variables=["question", "answer"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
#初始化题目数组
question_completed=[]
for item in question_with_answer:
    ans=chain.invoke({"question":item["content"],"answer":item["description"]})
    question_completed.append(ans)
    print(ans)

{'content': '在一次学校运动会上，男生和女生共参加了100米赛跑项目。如果女生人数是男生的2倍，而且总共有90人参加，那么男生和女生各有多少人参加了100米赛跑？', 'choices': ['男生30人，女生60人', '男生40人，女生50人', '男生20人，女生70人'], 'description': '设男生人数为x，则女生人数为2x。因为男生和女生总数为90，所以x + 2x = 90，解得x = 30。所以男生30人，女生60人。', 'correct_answer': '男生30人，女生60人'}
{'content': '小明和小红的储蓄罐里一共有360元。如果小明的钱是小红的3倍，那么小明和小红各有多少钱在他们的储蓄罐里？', 'choices': ['小明有270元，小红有90元', '小明有180元，小红有180元', '小明有240元，小红有120元'], 'description': '设小红有x元，则小明有3x元。因为他们一共有360元，所以x+3x=360，解得x=90。因此，小明有270元，小红有90元。', 'correct_answer': '小明有270元，小红有90元'}
{'content': '一个家庭在周末计划去超市购物，他们计划购买水果和蔬菜。如果水果的总价是蔬菜的4倍，而且他们一共花了500元，那么水果和蔬菜各花了多少钱？', 'choices': ['水果400元，蔬菜100元', '水果300元，蔬菜200元', '水果350元，蔬菜150元'], 'description': '设蔬菜的价格为x，那么水果的价格为4x。根据题意，我们有方程x + 4x = 500，解得x = 100，所以蔬菜花了100元，水果花了400元。', 'correct_answer': '水果400元，蔬菜100元'}


In [29]:
question_completed

[{'content': '小明上学的时间是早上8点15分，他从家里出发，步行到学校需要25分钟。请问，小明应该在什么时间离开家才能准时到达学校？',
  'choices': ['7:45 AM', '7:50 AM', '7:55 AM'],
  'description': '小明需要在8点15分到达学校，步行需要25分钟，所以他需要在7点50分离开家。',
  'correct_answer': '7:50 AM'},
 {'content': '妈妈准备了一盘饼干，共有60块。小华和小明分别吃了一些，小华吃了15块，小明吃了两倍于小华的数量。请问，盘子里还剩下多少块饼干？',
  'choices': ['A. 15块', 'B. 30块', 'C. 45块'],
  'description': '小华吃了15块，小明吃了两倍于小华的数量，即30块。他们一共吃了15+30=45块。所以，盘子里还剩下60-45=15块饼干。',
  'correct_answer': 'A. 15块'},
 {'content': '周末，爸爸带小红去公园玩。他们计划下午3点到达公园，玩3个小时后回家。但是，由于堵车，他们晚了30分钟到达公园。请问，他们应该在什么时间离开公园回家？',
  'choices': ['A. 6:00 PM', 'B. 6:30 PM', 'C. 7:00 PM'],
  'description': '由于他们晚了30分钟到达公园，原计划3点到达的变成了3点30分到达。计划在公园玩3个小时，即从3点30分开始玩到6点30分。所以他们应该在6点30分离开公园回家。',
  'correct_answer': 'B. 6:30 PM'}]

In [36]:
def format_question(question_data):
    formatted_text = f"{question_data['content']}\n"
    for i, choice in enumerate(question_data['choices'], start=65):
        formatted_text += f"{chr(i)}.  {choice}\n"
    formatted_text += f"正确答案：{chr(ord('A') + question_data['choices'].index(question_data['correct_answer']))}\n"
    formatted_text += f"解析：{question_data['description']}\n"
    return formatted_text


for question in question_completed:
    print(format_question(question))


在一次学校运动会上，男生和女生共参加了100米赛跑项目。如果女生人数是男生的2倍，而且总共有90人参加，那么男生和女生各有多少人参加了100米赛跑？
A.  男生30人，女生60人
B.  男生40人，女生50人
C.  男生20人，女生70人
正确答案：A
解析：设男生人数为x，则女生人数为2x。因为男生和女生总数为90，所以x + 2x = 90，解得x = 30。所以男生30人，女生60人。

小明和小红的储蓄罐里一共有360元。如果小明的钱是小红的3倍，那么小明和小红各有多少钱在他们的储蓄罐里？
A.  小明有270元，小红有90元
B.  小明有180元，小红有180元
C.  小明有240元，小红有120元
正确答案：A
解析：设小红有x元，则小明有3x元。因为他们一共有360元，所以x+3x=360，解得x=90。因此，小明有270元，小红有90元。

一个家庭在周末计划去超市购物，他们计划购买水果和蔬菜。如果水果的总价是蔬菜的4倍，而且他们一共花了500元，那么水果和蔬菜各花了多少钱？
A.  水果400元，蔬菜100元
B.  水果300元，蔬菜200元
C.  水果350元，蔬菜150元
正确答案：A
解析：设蔬菜的价格为x，那么水果的价格为4x。根据题意，我们有方程x + 4x = 500，解得x = 100，所以蔬菜花了100元，水果花了400元。

